In [ ]:
import pandas as pd
import numpy as np
from ARRrohm import get_df_extendida_ROHM, get_df_ROHM
from ARRrtra import get_df_extendida_RTRA, get_df_RTRA
from ARRdis import get_df_DIS, get_df_extendida_DIS
from functools import reduce

In [ ]:
# =============================
# PESOS DE LOS ÍNDICES
# =============================
pesos_ARR = {
    'ROHM': 3,
    'RTRA': 5,
    'RDIS': 5,
}
# =============================
# FUNCIÓN PARA CALCULAR ARR
# =============================
def calcular_ARR(row, pesos):
    valores = [row[col] * peso for col, peso in pesos.items() if col in row and pd.notna(row[col])]
    total_peso = sum(peso for col, peso in pesos.items() if col in row and pd.notna(row[col]))
    return sum(valores) / total_peso if total_peso > 0 else float("nan")

,SERIE,FECHA,ROHM
0,146916,2025-01-01,1.0
1,146916,2025-01-02,1.0
2,146916,2025-01-03,1.0
3,146916,2025-01-04,1.0
4,146916,2025-01-05,1.0


In [ ]:
# =============================
# FUNCIÓN PRINCIPAL
# =============================
def get_df_detalles_ARR():
    ROHM = get_df_ROHM()
    RTRA = get_df_RTRA()
    RDIS = get_df_DIS()
    tablas = {
        "ROHM": ROHM,
        "RTRA": RTRA,
        "RDIS": RDIS,
    }
    # Filtramos los que no sean None
    tablas = {k: v for k, v in tablas.items() if v is not None}

    if not tablas:
        return pd.DataFrame()
    
    # Merge progresivo de todas las tablas (asumimos que todas ya tienen SERIE + FECHA extendidos)
    resultado = reduce(
        lambda left, right: pd.merge(left, right, on=["SERIE", "FECHA"], how="outer"),
        tablas.values()
    )
    # Asegurar tipo datetime en FECHA
    resultado["FECHA"] = pd.to_datetime(resultado["FECHA"], errors="coerce")

    # Ordenamos por SERIE y FECHA DE MUESTRA
    resultado = resultado.sort_values(by=["SERIE", "FECHA"]).reset_index(drop=True)

    # ============================
    # Calcular ARR
    # ============================
    resultado["ARR"] = resultado.apply(lambda row: calcular_ARR(row, pesos_ARR), axis=1)

    # Selección de columnas finales
    columnas_finales = ["SERIE", "FECHA", "ARR"] + [col for col in tablas.keys()]
    resultado = resultado[columnas_finales]
    return resultado

,SERIE,FECHA,RTRA
0,146916,2025-01-01,1.0
1,146916,2025-01-02,1.0
2,146916,2025-01-03,1.0
3,146916,2025-01-04,1.0
4,146916,2025-01-05,1.0


In [ ]:
def get_df_detalles_ext_ARR():
    ROHM = get_df_extendida_ROHM()
    RTRA = get_df_extendida_RTRA()
    RDIS = get_df_extendida_DIS()
    tablas = {
        "ROHM": ROHM,
        "RTRA": RTRA,
        "RDIS": RDIS
    }
    # Filtramos los que no sean None
    tablas = {k: v for k, v in tablas.items() if v is not None}

    if not tablas:
        return pd.DataFrame()
    
    # Merge progresivo de todas las tablas (asumimos que todas ya tienen SERIE + FECHA extendidos)
    resultado = reduce(
        lambda left, right: pd.merge(left, right, on=["SERIE", "FECHA"], how="outer"),
        tablas.values()
    )
    # Asegurar tipo datetime en FECHA
    resultado["FECHA"] = pd.to_datetime(resultado["FECHA"], errors="coerce")

    # Ordenamos por SERIE y FECHA DE MUESTRA
    resultado = resultado.sort_values(by=["SERIE", "FECHA"]).reset_index(drop=True)

    # ============================
    # Calcular ARR
    # ============================
    resultado["ARR"] = resultado.apply(lambda row: calcular_ARR(row, pesos_ARR), axis=1)

    # Selección de columnas finales
    columnas_finales = ["SERIE", "FECHA", "ARR"] + [col for col in tablas.keys()]
    resultado = resultado[columnas_finales]
    return resultado

In [ ]:
df_ARR_detalles = get_df_detalles_ext_ARR()
df_ARR = df_ARR_detalles[['SERIE','FECHA','ARR']]


def get_df_extendida_ARR():
    return df_ARR
hola= get_df_detalles_ext_ARR()
print(get_df_detalles_ARR().head())
print(get_df_detalles_ext_ARR().tail())
# print(print(hola[(hola['FECHA'].dt.year == 2018) & (hola['FECHA'].dt.month == 6) & (hola['SERIE'] == "146660T3")]))
# # print(get_df_detalles_ext_ARR())

# print(df_ARR[(df_ARR['FECHA'].dt.year == 2018) & (df_ARR['FECHA'].dt.month == 6) & (df_ARR['SERIE'] == "146660T3")])

,SERIE,FECHA,ARR,ROHM,RTRA
0,146916,2025-01-01,1.0,1.0,1.0
1,146916,2025-01-02,1.0,1.0,1.0
2,146916,2025-01-03,1.0,1.0,1.0
3,146916,2025-01-04,1.0,1.0,1.0
4,146916,2025-01-05,1.0,1.0,1.0
...,...,...,...,...,...
796,146918,2025-09-20,1.0,1.0,NaN
797,146918,2025-09-21,1.0,1.0,NaN
798,146918,2025-09-22,1.0,1.0,NaN
799,146918,2025-09-23,1.0,1.0,NaN
